In [1]:
import tensorflow as tf
import numpy
import os
import json, pickle
import pandas
from functools import partial, reduce
from sklearn import manifold
import importlib

import sys
sys.path.append('../libs')

import flacdb
import prepare_data
import initialize
import data_pipeline
import conv_model
import plot_batch
import generate_report
import icd_util

In [2]:
! nvidia-smi

Mon May 11 13:06:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.00    Driver Version: 418.87.00    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX TIT...  On   | 00000000:05:00.0 Off |                  N/A |
| 22%   31C    P8    16W / 250W |      1MiB / 12212MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# ! cp -r /scr1/mimic/initial_data /scr1/mimic/initial_data_1464670

In [3]:
%%time

%matplotlib widget

plotter, model, X, Y, P, priors = generate_report.run(
    model_id = 1464849,
    checkpoint_index = -1,
    example_count_log2 = 14,
    dark = True
)
plotter()

found hypes ../hypes/1464849_20200509-214014.json 
found weights /scr1/checkpoints/1464849_20200509-214014_00384.ckpt
computing predictions
(14922, 90) predictions shape


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(SelectionSlider(continuous_update=False, description='Condition:', layout=Layout(width='…

CPU times: user 3min 34s, sys: 44.4 s, total: 4min 19s
Wall time: 2min 56s


<function generate_report.get_plotter.<locals>.update(code, threshold, example_index, log_scale)>

In [5]:
metadata = pandas.read_hdf('/scr-ssd/mimic/metadata.hdf')

In [6]:
metadata

hadm_id  subject_id                    time gender        dob  \
rec_id  seg_id                                                                  
3000003 1        173995        9950 2155-04-19 19:44:07.664      M 2108-02-01   
        2        173995        9950 2155-04-19 19:46:22.685      M 2108-02-01   
        3        173995        9950 2155-04-19 19:46:25.757      M 2108-02-01   
        4        173995        9950 2155-04-19 19:46:33.981      M 2108-02-01   
        5        173995        9950 2155-04-19 19:46:37.053      M 2108-02-01   
...                 ...         ...                     ...    ...        ...   
3999988 6            -1          -1                     NaT    NaN        NaT   
        7            -1          -1                     NaT    NaN        NaT   
        8            -1          -1                     NaT    NaN        NaT   
        9            -1          -1                     NaT    NaN        NaT   
        10           -1          -1                     NaT    NaN        NaT   

                age  weight  height  sig_len death_time   race  
rec_id  seg_id                                                  
3000003 1        47     NaN      74    16878        NaT  white  
        2        47     NaN      74      384        NaT  white  
        3        47     NaN      74     1028        NaT  white  
        4        47     NaN      74      384        NaT  white  
        5        47     NaN      74   438826        NaT  white  
...             ...     ...     ...      ...        ...    ...  
3999988 6        -1     NaN      -1     5250        NaT    NaN  
        7        -1     NaN      -1      250        NaT    NaN  
        8        -1     NaN      -1      500        NaT    NaN  
        9        -1     NaN      -1      250        NaT    NaN  
        10       -1     NaN      -1    24875        NaT    NaN  

[2273298 rows x 11 columns]

In [3]:
from matplotlib import pyplot
pyplot.style.use('dark_background')

In [8]:
organs = {
    'brain': [
        '191', '198.3', '348.1', '348.31', '348.4', '348.5', '349.82', 
        '430-438', '430', '431', '432.1', '434', '437.3', '850-854', '852'
    ],
    'liver': [
        '070', '155', '155.0', '570', '571', '571.1', '571.2', '572', '572.2',
        '572.3', '572.4', '574'
    ],
    'lung': ['162', '480-488', '507', '511', '518.0', '518.81', '997.3'],
    'kidney': ['403', '580-589', '584', '585'],
    'heart': [
        '396', '397.0', '410-414', '410', '410.7', '414.0', '416', '424.0', 
        '424.1', '425', '426', '427', '427.1', '427.31', '427.32', '427.41', 
        '427.5', '428', '428.0', '428.2', '428.3', '785.51', '997.1'
    ],
    'sepsis': ['038', '785.52', '995.9', '995.92']
}

colors = {
    'brain': 'green',
    'liver': 'gold',
    'lung': 'blue', 
    'kidney': 'magenta',
    'heart': 'red',
    'sepsis': 'peru'
}

def lookup_color(code):
    for k in organs:
        if code in organs[k]:
            return colors[k]
    else:
        return 'white'

In [9]:
group_names = icd_util.load_group_strings()

def get_name(code):
    if code not in group_names:
        return code.replace('_', ' ').title()
    name = code + ': ' + group_names[code]
    name = name.replace('/', '_')
    for character in "',()[]":
        name = name.replace(character, '')
    return name

names = [get_name(i) for i in priors.index]

In [13]:
from matplotlib import pyplot
pyplot.style.use('default')

In [50]:
# z = manifold.TSNE(perplexity=25, n_iter=1000).fit_transform(P.T)
fig = pyplot.figure(7)
fig.clear()
c = [lookup_color(i) for i in priors.index]
pyplot.scatter(z[:, 0], z[:, 1], c=c, s=64, linewidths=1, edgecolors='black')
ax = fig.gca()
for i in ax.spines.values():
    i.set_visible(False)
pyplot.xticks([])
pyplot.yticks([])
texts = [
    pyplot.text(z[i, 0], z[i, 1], ' ' * 4 + txt, ha='left', va='center', fontsize=6) 
    for i, txt in enumerate(names)
]
fig.canvas.layout.width = '1600px'
fig.canvas.layout.height = '1200px'
fig.canvas

Canvas(layout=Layout(height='1200px', width='1600px'), toolbar=Toolbar(toolitems=[('Home', 'Reset original vie…

In [54]:
import pickle
with open('embedding.pkl', 'wb') as f:
    pickle.dump([P, z], f)

In [57]:
P_, z_ = pickle.load(open('embedding.pkl', 'rb'))
assert((z_ == z).all())

In [16]:
import matplotlib
import matplotlib.cm as colormap

In [101]:
fig = pyplot.figure(16)
fig.clear()
fig.canvas.layout.width = '1600px'
fig.canvas.layout.height = '1200px'
i = numpy.random.randint(len(P))
p = P[i]
y = Y['diagnosis'][i]

norm = matplotlib.colors.Normalize(vmin=0, vmax=1, clip=True)
mapper = colormap.ScalarMappable(norm=norm, cmap=colormap.Reds)
c = mapper.to_rgba(p)
c[:, -1] = p

# edges = ['white' if i == 1 else 'black' for i in y]
edges = ['white' if i != 1 else 'black' for i in y]
pyplot.scatter(z[:, 0], z[:, 1], c=c, s=80, linewidths=2, edgecolors=edges)
ax = fig.gca()
for spine in ax.spines.values():
    spine.set_visible(False)
pyplot.xticks([])
pyplot.yticks([])
texts = [
    pyplot.text(z[j, 0], z[j, 1], ' ' * 4 + names[j], ha='left', va='center', fontsize=6, alpha=p[j]) 
    for j in range(len(p))
]
fig.canvas

Canvas(layout=Layout(height='1200px', width='1600px'), toolbar=Toolbar(toolitems=[('Home', 'Reset original vie…

In [100]:
fig.canvas

Canvas(layout=Layout(height='1200px', width='1600px'), toolbar=Toolbar(toolitems=[('Home', 'Reset original vie…

In [24]:
p = P[0]
o = numpy.zeros(len(p))
l = numpy.ones(len(p))
colors = numpy.stack([p, o, o, l]).T
colors.shape

(96, 4)

In [19]:
import matplotlib
import matplotlib.cm as colormap
# from matplotlib import cm as colormap

lst = [1.9378076554115014, 1.2084586588892861, 1.2133096565896173, 1.2427632053442292, 
       1.1809971732733273, 0.91960143581348919, 1.1106310149587162, 1.1106310149587162, 
       1.1527004351293346, 0.87318084435885079, 1.1666132876686799, 1.1666132876686799]

minima = min(lst)
maxima = max(lst)

norm = matplotlib.colors.Normalize(vmin=0, vmax=1, clip=True)
mapper = colormap.ScalarMappable(norm=norm, cmap=colormap.Reds)

for v in lst:
    print(mapper.to_rgba(v))

(0.403921568627451, 0.0, 0.05098039215686274, 1.0)
(0.403921568627451, 0.0, 0.05098039215686274, 1.0)
(0.403921568627451, 0.0, 0.05098039215686274, 1.0)
(0.403921568627451, 0.0, 0.05098039215686274, 1.0)
(0.403921568627451, 0.0, 0.05098039215686274, 1.0)
(0.556478277585544, 0.03690888119953864, 0.07066512879661667, 1.0)
(0.403921568627451, 0.0, 0.05098039215686274, 1.0)
(0.403921568627451, 0.0, 0.05098039215686274, 1.0)
(0.403921568627451, 0.0, 0.05098039215686274, 1.0)
(0.6476432141484045, 0.05896193771626297, 0.08247597078046903, 1.0)
(0.403921568627451, 0.0, 0.05098039215686274, 1.0)
(0.403921568627451, 0.0, 0.05098039215686274, 1.0)
